In [ ]:

import os
import random
import shutil
import sys

import cv2
import dlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from mtcnn import MTCNN

# Face Detection

In [ ]:
# Base directory where the emotion folders are located
base_dir = "Datasets/RAF-FER-SFEW-AN"
# New dataset directory
new_dataset_dir = "Datasets/combined_dataset_faces_only_mtcnn"

# Emotion classes
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Create directories for the new dataset
if not os.path.exists(new_dataset_dir):
    os.makedirs(new_dataset_dir)
for emotion in emotions:
    os.makedirs(os.path.join(new_dataset_dir, emotion), exist_ok=True)
    
for emotion in emotions:
    emotion_dir = os.path.join(base_dir, emotion)
    all_images = os.listdir(emotion_dir)
    existing_dir = os.path.join(new_dataset_dir, emotion)
    existing_images = os.listdir(existing_dir)
    all_images = [img for img in all_images if img not in existing_images]
    
    for image in all_images:
        img_path = os.path.join(emotion_dir, image)
        try:
            img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            detector = MTCNN()
            
            # Increment correct count if prediction matches the folder name
            if detector.detect_faces(img):
                # Copy the correctly identified image to the new dataset
                shutil.copy(img_path, os.path.join(new_dataset_dir, emotion, image))
            
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# Base Model - Data Preparation
Face Detection
Image Cropping, Alignment and Grayscaling
Rescaling and Padding
128, 128, 1 grayscale images

This code only needs to be run once to create the dataset used for training.

In [ ]:
# Initialize MTCNN detector
detector = MTCNN()

base_dir = "Datasets/combined_dataset_faces_only_mtcnn"
processed_folder = "Datasets/combined_dataset_processed_128_1"

# Emotion classes
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


# Create directories for the new dataset
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)
for emotion in emotions:
    os.makedirs(os.path.join(processed_folder, emotion), exist_ok=True)

target_width, target_height = 128, 128

for emotion in emotions:
    emotion_dir = os.path.join(base_dir, emotion)
    all_images = os.listdir(emotion_dir)
    existing_dir = os.path.join(processed_folder, emotion)
    existing_images = os.listdir(existing_dir)
    all_images = [img for img in all_images if img not in existing_images]

    for image_name in all_images:
        try:
            img_path = os.path.join(emotion_dir, image_name)
            # Load an image
            img = cv2.imread(img_path)
            image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Detect faces in the image
            results = detector.detect_faces(image_rgb)
            
            if results:
                # Extract the bounding box and facial landmarks
                x, y, width, height = results[0]['box']
                landmarks = results[0]['keypoints']
                
                # Calculate the angle to align eyes horizontally
                left_eye = landmarks['left_eye']
                right_eye = landmarks['right_eye']
                delta_x = right_eye[0] - left_eye[0]
                delta_y = right_eye[1] - left_eye[1]
                angle = np.arctan(delta_y / delta_x) * 180 / np.pi
                
                # Center coordinates of the face
                center = (x + width // 2, y + height // 2)
                # Rotation matrix
                rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
                # Perform the rotation on the entire image
                aligned_image = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))
                # Crop the face
                aligned_face = aligned_image[y:y + height, x:x + width]
                # Convert to grayscale
                aligned_face_gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
                
                # Convert the aligned grayscale face to PIL Image for resizing and padding
                pil_image = Image.fromarray(aligned_face_gray)
                
                # Calculate the scaling factor to maintain aspect ratio
                aspect_ratio = min(target_width / pil_image.width, target_height / pil_image.height)
                new_width = int(pil_image.width * aspect_ratio)
                new_height = int(pil_image.height * aspect_ratio)
                # Resize the image with the scaling factor
                pil_image = pil_image.resize((new_width, new_height), Image.LANCZOS)
                
                # Create a new image with padding
                new_image = Image.new("L", (target_width, target_height), (0))
                paste_position = ((target_width - new_width) // 2, (target_height - new_height) // 2)
                new_image.paste(pil_image, paste_position)
                
                save_path = os.path.join(processed_folder, emotion, image_name)
                new_image.save(save_path)
                
        except Exception as e:
            print(f"Error processing {img_path}: {e}")


# Transfer Learning Model - Data Preparation
Face Detection
Image Cropping, Alignment and Grayscaling
Rescaling and Padding
224, 224, 3 grayscale images as expected by the pretrained models

This code only needs to be run once to create the dataset used for training.

In [ ]:
# Initialize MTCNN detector
detector = MTCNN()

base_dir = "Datasets/combined_dataset_faces_only_mtcnn"
processed_folder = "Datasets/combined_dataset_processed_224_3"

# Emotion classes
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


# Create directories for the new dataset
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)
for emotion in emotions:
    os.makedirs(os.path.join(processed_folder, emotion), exist_ok=True)

target_width, target_height = 224, 224

for emotion in emotions:
    emotion_dir = os.path.join(base_dir, emotion)
    all_images = os.listdir(emotion_dir)
    existing_dir = os.path.join(processed_folder, emotion)
    existing_images = os.listdir(existing_dir)
    all_images = [img for img in all_images if img not in existing_images]

    for image_name in all_images:
        try:
            img_path = os.path.join(emotion_dir, image_name)
            # Load an image
            img = cv2.imread(img_path)
            image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Detect faces in the image
            results = detector.detect_faces(image_rgb)
            
            if results:
                # Extract the bounding box and facial landmarks
                x, y, width, height = results[0]['box']
                landmarks = results[0]['keypoints']
                
                # Calculate the angle to align eyes horizontally
                left_eye = landmarks['left_eye']
                right_eye = landmarks['right_eye']
                delta_x = right_eye[0] - left_eye[0]
                delta_y = right_eye[1] - left_eye[1]
                angle = np.arctan(delta_y / delta_x) * 180 / np.pi
                
                # Center coordinates of the face
                center = (x + width // 2, y + height // 2)
                # Rotation matrix
                rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
                # Perform the rotation on the entire image
                aligned_image = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))
                # Crop the face
                aligned_face = aligned_image[y:y + height, x:x + width]
                # Convert to grayscale
                aligned_face_gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
                
                # Convert the aligned grayscale face to PIL Image for resizing and padding
                pil_image = Image.fromarray(aligned_face_gray)
                
                # Calculate the scaling factor to maintain aspect ratio
                aspect_ratio = min(target_width / pil_image.width, target_height / pil_image.height)
                new_width = int(pil_image.width * aspect_ratio)
                new_height = int(pil_image.height * aspect_ratio)
                # Resize the image with the scaling factor
                pil_image = pil_image.resize((new_width, new_height), Image.LANCZOS)
                
                # Create a new image with padding
                new_image = Image.new("L", (target_width, target_height), (0))
                paste_position = ((target_width - new_width) // 2, (target_height - new_height) // 2)
                new_image.paste(pil_image, paste_position)
                
                # Convert the single-channel grayscale image to 3-channel
                new_image_rgb = new_image.convert("RGB")
                
                save_path = os.path.join(processed_folder, emotion, image_name)
                new_image_rgb.save(save_path)
                
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# Landmark Model - Determine Landmarks
Create files with 6 channels, 1 for grayscale image and 5 for facial features as NPY file. Eyebrows and eyes are combined into single channels since assysmetric features are not relevant in this model with this dataset.

In [ ]:
# Define paths
data_dir = 'Datasets/combined_dataset_processed_128_1'
new_data_dir = 'Datasets/combined_dataset_enhanced_npy_6_channels'
shape_predictor_path = 'PretrainedModels/shape_predictor_68_face_landmarks.dat'

if not os.path.exists(new_data_dir):
    os.makedirs(new_data_dir)

# Initialize Dlib's detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

def extract_landmarks_and_save(image_path, predictor, new_data_dir):
    # Load the original image and prepare output directory and path
    class_dir, image_name = os.path.split(image_path)
    _, class_name = os.path.split(class_dir)
    new_class_dir = os.path.join(new_data_dir, class_name)
    if not os.path.exists(new_class_dir):
        os.makedirs(new_class_dir)
    # Adjust the file extension for saving as .npy
    new_image_path = os.path.join(new_class_dir, image_name.replace('.jpg', '.npy'))

    # Load the image using OpenCV
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Prepare an empty list to hold feature images
    feature_images = [gray_image]  # Start with the grayscale image

    # Assuming the image is a cropped face, create a rectangle covering the whole image
    dlib_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for dlib
    rect = dlib.rectangle(0, 0, gray_image.shape[1]-1, gray_image.shape[0]-1)
    landmarks = predictor(dlib_img, rect)

    # Define indices for facial features
    feature_indices = {
        'jawline': list(range(0, 17)),
        # 'right_eyebrow': list(range(17, 22)),
        # 'left_eyebrow': list(range(22, 27)),
        'nose': list(range(27, 36)),
        # 'right_eye': list(range(36, 42)),
        # 'left_eye': list(range(42, 48)),
        'mouth': list(range(48, 68))
        # 'eyebrows': range(17, 27),  # Combining left and right eyebrows
        # 'eyes': range(36, 48),  # Combining left and right eyes
    }

    # Create and draw feature images
    for feature, indices in feature_indices.items():
        # Initialize a single-channel image (black)
        feature_img = np.zeros_like(gray_image)
        points = np.array([[landmarks.part(n).x, landmarks.part(n).y] for n in indices], dtype=np.int32).reshape((-1, 1, 2))

        # Use polylines to draw the feature
        cv2.polylines(feature_img, [points], isClosed=(feature != 'jawline'), color=255, thickness=1)
        feature_images.append(feature_img)
    
    # Function to create feature image for separate features without connecting them
    def create_feature_image(indices, gray_image):
        feature_img = np.zeros_like(gray_image)
        for n in indices:
            x, y = landmarks.part(n).x, landmarks.part(n).y
            cv2.circle(feature_img, (x, y), 1, 255, -1)  # Use cv2.circle to mark each point
        return feature_img
    
    # Create temporary images for each eye and eyebrow
    left_eyebrow_img = create_feature_image(list(range(22, 27)), gray_image)
    right_eyebrow_img = create_feature_image(list(range(17, 22)), gray_image)
    left_eye_img = create_feature_image(list(range(42, 48)), gray_image)
    right_eye_img = create_feature_image(list(range(36, 42)), gray_image)
    
    # Combine temporary images for eyebrows and eyes into single images
    combined_eyebrows_img = cv2.bitwise_or(left_eyebrow_img, right_eyebrow_img)
    combined_eyes_img = cv2.bitwise_or(left_eye_img, right_eye_img)
    
    # Append the combined images to the feature_images list
    feature_images.append(combined_eyebrows_img)
    feature_images.append(combined_eyes_img)

    # Stack the single-channel images to form a multi-channel image
    multi_channel_image = np.stack(feature_images, axis=-1)

    # Save the multi-channel image as NPY
    np.save(new_image_path, multi_channel_image)

def process_directory_with_landmarks(directory, new_data_dir, predictor):
    total_files = sum([len(files) for r, d, files in os.walk(directory)])
    processed_files = 0
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                extract_landmarks_and_save(image_path, predictor, new_data_dir)
                processed_files += 1
                progress_percentage = (processed_files / total_files) * 100
                print(f"\rProcessed {processed_files}/{total_files} files ({progress_percentage:.2f}%)", end="")
                sys.stdout.flush()
    print("\nFinished processing all files.")

# Process the directory and save images with additional channels
process_directory_with_landmarks(data_dir, new_data_dir, predictor)


### Visualisation of the generated files for the landmark model

In [ ]:
def display_image_and_channels(image_data):
    n_channels = image_data.shape[2]
    fig, axs = plt.subplots(1, n_channels, figsize=(15, 5))
    
    for i in range(n_channels):
        # For grayscale visualization, ensure the single channel is repeated 3 times
        if i == 0:  # Assuming the first channel is the grayscale original image
            axs[i].imshow(image_data[:, :, i], cmap='gray')
        else:
            # Visualize the additional channels, which represent the facial features
            # Here, a mask is used to only display the areas of interest
            mask = image_data[:, :, i] > 0  # Creating a mask where the feature is drawn
            display_img = np.zeros(image_data[:, :, 0].shape + (3,), dtype=np.uint8)  # Prepare a blank RGB image
            display_img[..., 1][mask] = 255  # Draw the feature in green on the mask
            axs[i].imshow(display_img)
        axs[i].axis('off')  # Hide axes for better visualization

    plt.show()

def load_and_display_random_images(directory, num_images=15):
    # List all .npy files in the specified directory
    all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.npy')]
    
    # Select a random subset of files
    selected_files = random.sample(all_files, min(len(all_files), num_images))
    
    # Load and display each selected file
    for file_path in selected_files:
        print(f"Displaying: {file_path}")
        image_data = np.load(file_path)
        display_image_and_channels(image_data)

# Specify the directory containing the .npy files for the "happy" class
npy_directory = 'Datasets/combined_dataset_enhanced_npy/happy/'

# Load and display 15 random images and their channels
load_and_display_random_images(npy_directory, num_images=5)